In [2]:
import numpy as np
from numpy.linalg import lstsq

## Let's start by making a regular old dot product of two matrices to see how well the ordinary least squares solver works at returning the original matrices when we know one of them.

In [3]:
a = np.array([[11,13,12],[14,16,15],[10,18,19]]) 
print(a)

[[11 13 12]
 [14 16 15]
 [10 18 19]]


In [4]:
x = np.array([[1,2,3],[6,5,4],[9,7,8]])
print(x)

[[1 2 3]
 [6 5 4]
 [9 7 8]]


In [5]:
b = a.dot(x)
print(b)

[[197 171 181]
 [245 213 226]
 [289 243 254]]


In [6]:
for l, arr in zip(['a','x','b'],[a,x,b]):
    print('{}:\n'.format(l), arr, '\n\n')

a:
 [[11 13 12]
 [14 16 15]
 [10 18 19]] 


x:
 [[1 2 3]
 [6 5 4]
 [9 7 8]] 


b:
 [[197 171 181]
 [245 213 226]
 [289 243 254]] 




## **A** $\cdot$ **X** = **B**
## How do we get **X** if we only have **A** and **B**? 

In [7]:
lstsq?

Signature: lstsq(a, b, rcond='warn')
Docstring:
Return the least-squares solution to a linear matrix equation.

Solves the equation `a x = b` by computing a vector `x` that
minimizes the Euclidean 2-norm `|| b - a x ||^2`.  The equation may
be under-, well-, or over- determined (i.e., the number of
linearly independent rows of `a` can be less than, equal to, or
greater than its number of linearly independent columns).  If `a`
is square and of full rank, then `x` (but for round-off error) is
the "exact" solution of the equation.

Parameters
----------
a : (M, N) array_like
    "Coefficient" matrix.
b : {(M,), (M, K)} array_like
    Ordinate or "dependent variable" values. If `b` is two-dimensional,
    the least-squares solution is calculated for each of the `K` columns
    of `b`.
rcond : float, optional
    Cut-off ratio for small singular values of `a`.
    For the purposes of rank determination, singular values are treated
    as zero if they are smaller than `rcond` times the large

In [8]:
lstsq(a,b)[0]

/Users/mark.llorente/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


array([[1., 2., 3.],
       [6., 5., 4.],
       [9., 7., 8.]])

## **A** $\cdot$ **X** = **B**
## Now, how do we get _**A**_ if we only have **X** and **C**? 

In [9]:
print(lstsq(x.T, b.T, rcond=None)[0])


[[11. 14. 10.]
 [13. 16. 18.]
 [12. 15. 19.]]


In [10]:
a_approx = lstsq(x.T,b.T, rcond=None)[0].T
print(a_approx)
x_approx = lstsq(a, b, rcond=None)[0]
print(x_approx)

[[11. 13. 12.]
 [14. 16. 15.]
 [10. 18. 19.]]
[[1. 2. 3.]
 [6. 5. 4.]
 [9. 7. 8.]]


## It works!
Because we knew the exact matrices that made B, we know that A and X in the above examples should be exactly solvable! 
## Side Note: 
x_approx will be slightly off from x and a_approx from a, so if you try to check validity of the solution by using the comparison method with the original a or x matrices...

In [11]:
print(a==a_approx)
print(a-a_approx)
print(x-x_approx)

[[False False False]
 [False False False]
 [False False False]]
[[ 2.30926389e-14  3.55271368e-15 -1.06581410e-14]
 [ 5.86197757e-14  3.19744231e-14 -4.08562073e-14]
 [ 7.10542736e-15  7.10542736e-15 -3.55271368e-15]]
[[-1.88737914e-14 -2.04281037e-14  1.55431223e-14]
 [ 8.79296636e-14  8.17124146e-14 -6.03961325e-14]
 [-6.39488462e-14 -5.95079541e-14  5.32907052e-14]]


Not a real problem :) 
Rounding to 12 decimal places out:

In [12]:
print(np.round(a-a_approx,12))

[[ 0.  0. -0.]
 [ 0.  0. -0.]
 [ 0.  0. -0.]]


After some very mild rounding:

In [13]:
print(a==np.round(a_approx,12))

[[ True  True  True]
 [ True  True  True]
 [ True  True  True]]


### Why use this tool? 
It doesn't just work on exact factors. We can use it to find approximations for matrices that could multiply to make the original matrix with minimal reconstruction error.

Say we have **W**$\cdot$**H**=**V** and we know **W** and **V** but want **H**. What's a good approximation for it?

In [14]:
V = np.random.randint(0,5,[4,6])
print(V)

[[1 1 2 3 0 2]
 [2 2 1 0 0 2]
 [3 3 4 3 1 1]
 [3 0 0 0 3 0]]


In [15]:
W = np.array([[1,0,2],[4,2,0],[3,1,2],[2,4,9]])
print(W)

[[1 0 2]
 [4 2 0]
 [3 1 2]
 [2 4 9]]


In [16]:
H = lstsq(W,V,rcond=None)[0]
print(H)

[[ 0.75661376  1.04232804  1.3968254   1.28571429 -0.05291005  0.77248677]
 [-0.4021164  -0.97354497 -2.12698413 -2.57142857  0.21693122 -0.76719577]
 [ 0.34391534  0.2010582   0.63492063  0.85714286  0.24867725  0.16931217]]


### Interesting 
We have some negative values in this H matrix. Might be worth using the `numpy.ndarray.clip()` method in the future.

In [18]:
H.clip(min=.000005)

array([[7.56613757e-01, 1.04232804e+00, 1.39682540e+00, 1.28571429e+00,
        5.00000000e-06, 7.72486772e-01],
       [5.00000000e-06, 5.00000000e-06, 5.00000000e-06, 5.00000000e-06,
        2.16931217e-01, 5.00000000e-06],
       [3.43915344e-01, 2.01058201e-01, 6.34920635e-01, 8.57142857e-01,
        2.48677249e-01, 1.69312169e-01]])